In [41]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
#eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
dopout = 0.2

cuda


In [34]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [35]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data=torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [36]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [58,  1, 69, 65, 58, 54, 72, 58],
        [67, 57,  1, 65, 68, 68, 64, 62],
        [ 3, 44, 61, 58, 78,  1, 54, 71]], device='cuda:0')
targets:
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [ 1, 69, 65, 58, 54, 72, 58, 57],
        [57,  1, 65, 68, 68, 64, 62, 67],
        [44, 61, 58, 78,  1, 54, 71, 58]], device='cuda:0')


In [37]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [38]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


5;jSa&Yo,qBI1r"K5E&na4BW"0;[[﻿4dh[IK5Wt:Rr9jH?z jINeH50X&"GqYQ'A3EysvMmx5Xuh3q Z.QH
WshN
Yg]PP﻿;'PlIXllI9"OqSa_p]jEmp_X'YG'_RUS*HjfL7M09i9'M6Ou3Nla"tIXHWtiOqZcnpmNADI_XOrQ;r9:cEmAdxKn!.0DS2mrGx8)j8VK:l03H.072a)Z8HO_*5CuF8iHZNAeH5GFBRUw]20qWla﻿"Za&,1y7H2YrIK
r*K,_R5Q:;nX0:]Wi!ZvRqgcJ2Nt-0Dgq[bl 6JIxm?wFN&s":YH[dFiY﻿)bEv,fXCBkZJEy;
[9kAVs-GpOp0(*yPW3Wc,odG2cJ_:p,_Bkh?3W﻿aJS3W*!DlC7[Q4 pADS'uy5W"thOl0z)cQ4njg:RqW;nM﻿I!9(qk*&.pA6*vM9kgbCm﻿::7HW KXj JOodJxmdqBQ!]PW:cro9EmHL8﻿hrbqr"&,:]&,AsYiB9jh3)snX


In [42]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 3.197, val loss: 3.238
step: 250, train loss: 3.157, val loss: 3.174
step: 500, train loss: 3.148, val loss: 3.175
step: 750, train loss: 3.103, val loss: 3.139
3.2729604244232178


***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the 
optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a 
continuous output. It measures the average squared difference between the predicted and actual values, and is often used to 
train neural networks for regression tasks.
2. **Gradient Descent (GD):** is an optimization algorithm used to minimize the loss function of a machine learning model. The 
loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is 
to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth 
out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or 
varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning 
rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average 
of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default 
optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to 
regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the 
properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [40]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"."F( ZPQn;BC?Mg'bJh?*(JS
3
v,PUSpu . cJ2*K-] hng'ch0jgewnk4CG&ea-Sj&TGoy?ua4qEvMfyointhow,uG2eoQndMOW
HOQYRZ
u!_i5KD&reode i
rl v(EALJ﻿GP&Yr.iVCZOheDth du  r
miy2gt th?rdi,2YubunenH.0TJFmf,u[oYqEvoQUGa&tGPR
":X
bjU,_ tr fb05F?ll,&z  wne roswskl DIty d;LxF2.'D'?WdHhPv[Tw]&Zj]MBz,zEadGR)2BgM
RK7xaAKVkFg33XJ33]AakXS]]8&nh[[mcrxr

p dW﻿)BvjGP,'tu,BsgsWb(gi&]yicq_'s*"yugzxp."7;SDSQHrr
"n:6J[KnH2'n?yo?RSc3bQH_D90GAUSjVLS2MObWb0Xfor
m r  pa]jJ!27
hN. O0*H﻿?CINAe e Iy hX[jV_mib(JPsJ:7fl_psk?(_
d;O!,2dz
